In [2]:
import osmnx as ox
import momepy
import geopandas as gpd
import networkx
from osmnx import io
from geopandas import GeoDataFrame, GeoSeries
from centerline.geometry import Centerline
path ='output'
project_crs = 'epsg:3857'
import pandas as pd
import ast
from sklearn.cluster import DBSCAN
from shapely.geometry import Polygon,Point,LineString,MultiPolygon

### Run only when download for first time

In [7]:
# Create graph from OSM server and project it
graph = ox.graph_from_place('Torino', network_type='all')
graph  = ox.bearing.add_edge_bearings(graph, precision=1)
graph_pro = ox.projection.project_graph(graph, to_crs=project_crs)
io.save_graph_geopackage(graph_pro, filepath='.', encoding='utf-8', directed=False)

### End

In [1]:
# delete segments without names
my_gdf = gpd.read_file(f'{path}/edges.shp')
not_null= my_gdf.dropna(subset='name')
not_null.to_file(f'{path}/not_null.shp')

NameError: name 'gpd' is not defined

In [73]:
# delete segments present any kind of roundabout or tunnel
not_roundaout = not_null[~((not_null['junction']=='roundabout')|(not_null['junction']=='circular')|(not_null['tunnel']=='building_passage')|(not_null['tunnel']=='yes')| (not_null['highway']=='cycleway')| (not_null['highway']=='path'))]
not_roundaout.to_file(f'{path}/not_roundabout_tunnel.shp')

In [75]:
# Project dataframe and calculate angle (0 to 180)
df_pro = not_roundaout.to_crs(project_crs)
df_pro['angle'] = df_pro['bearing'].apply(lambda x:x if x<180 else x-180)
df_pro.to_file(f'{path}/pro.shp')

In [ ]:
# only the first value will be saed highway with more than 1 value

In [4]:
def length_of_parallel(my_s_join: GeoDataFrame, the_buffer:GeoSeries, geo_field:str)-> int:
    my_s_join['geometry'] =my_s_join[geo_field]
    new_data_0 = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs= project_crs), how='inner').reset_index()
    if len(new_data_0)==0:
        return 0
    return len(new_data_0[new_data_0['index']!= new_data_0['index_right']])

In [5]:
def check_parallelism(to_translate: GeoDataFrame, is_test:bool=False)->bool:
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x:x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x:x.parallel_offset(35, 'left'))
    if is_test:
        to_translate.drop(columns= ['geometry', new_geometry[0]]).rename(columns = {new_geometry[1]: 'geometry'}).to_file(f'{path}/res_translate_{new_geometry[1]}.shp')
        to_translate.drop(columns= ['geometry', new_geometry[1]]).rename(columns = {new_geometry[0]: 'geometry'}).to_file(f'{path}/res_translate_{new_geometry[0]}.shp')
        my_buffer.to_file(f'{path}/buffers_test.shp')
    if length_of_parallel(to_translate, my_buffer, 'geometry_right') >0 or length_of_parallel(to_translate, my_buffer, 'geometry_left') >0:
        return True
    else:
        return False

In [6]:
def simplify(x):
    simplify_poly = one_buffer.simplify(x, preserve_topology=False)
    if simplify_poly.area==0:
        simplify_poly= simplify(x/2)
    return simplify_poly



In [7]:
df_pro = gpd.read_file((f'{path}/pro.shp'))

In [8]:
# Main point to start



my_groupby = df_pro.groupby('name')
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': []}
for_time = len(my_groupby)
for i, street in enumerate(my_groupby):
    # Calculate time to run
    print(round(i / for_time * 100, 2))
    res = street[1]
    name = street[0]
    # groupby angle
    res =  res.dropna(subset= ['angle'],axis=0)
    if len(res)==0:
        continue
    res['group'] = DBSCAN(eps=5, min_samples=2).fit(res['angle'].to_numpy().reshape(-1, 1)).labels_
    cur_group = res[res['group'] > -1].groupby('group')
    for group in cur_group:
        data = group[1]
        if check_parallelism(data, is_test=False):
            # if among of lines with same angles some are parallel,find the center line for each group
            for group in cur_group:
                data = group[1]
                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                # buffers.to_file(f'{path}/buffers.shp')
                one_buffer = buffers.unary_union
                simplify_poly = simplify(100)
                if isinstance(simplify_poly, MultiPolygon):
                    pnt_list = []
                    for polygon in simplify_poly:
                        pnt_list.extend(polygon.exterior.coords[:-1])
                else:
                    pnt_list = simplify_poly.exterior.coords[:-1]
                list_shp = [Point(item) for item in pnt_list]
                dis = 0
                for i, point in enumerate(list_shp):
                    j = i
                    for point2 in list_shp[i + 1:]:
                        j += 1
                        if point.distance(point2) > dis:
                            max_dis = (i, j)
                            dis = point.distance(point2)
                dic_final['name'].append(name)
                dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]], pnt_list[max_dis[1]])))
                dic_final['highway'].append(data.iloc[0]['highway'])
                dic_final['bearing'].append(data['angle'].mean())
GeoDataFrame(dic_final, crs='epsg:3857').to_file(f'{path}/one_line.shp')

0.0
0.04
0.08
0.12


C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iterat

0.16
0.2
0.24
0.28


C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iterat

0.32
0.37


...

1.87
1.91
1.95
1.99
2.03
2.07


C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iterat

2.11
2.15
2.19


C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:
C:\Users\Achituv\AppData\Local\Temp\ipykernel_20092\3007926787.py:31: ShapelyDeprecationWarning: Iterat

KeyboardInterrupt: 

In [32]:
list(dic_final.keys())



['name', 'geometry', 'highway', 'bearing']

In [37]:
# create netwrok
new_network = df_pro[~df_pro['name'].isin(dic_final['name'])][list(dic_final.keys())]
new_network['is_simplify'] = 0
simplify = GeoDataFrame(dic_final, crs='epsg:3857')
simplify['is_simplify'] = 1
new_network.append(simplify).to_file(f'{path}/new_network.shp')

C:\Users\Achituv\AppData\Local\Temp\ipykernel_22516\3334121598.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_network.append(simplify).to_file(f'{path}/new_network.shp')
C:\Users\Achituv\AppData\Local\Temp\ipykernel_22516\3334121598.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_network.append(simplify).to_file(f'{path}/new_network.shp')


In [ ]:
dic_final = {'name':[],'geometry':[], 'highway':[],'bearing':[]}
for_time = len(my_groupby)
for i,street in enumerate(my_groupby):
    # Calculate time to run
    print(round(i/for_time*100,2))
    res= street[1]
    name = street[0]
    # groupby highway and angle
    groupbyhighway = res.groupby('highway')
    the_largest_street = res.groupby('highway').sum()['length'].sort_values(ascending=False).index[0]
    for_bearing_0 = groupbyhighway.get_group(the_largest_street).reset_index()
    for_bearing= for_bearing_0.dropna(subset= ['angle'],axis=0)
    if len(for_bearing)==0:
        continue
    for_bearing['group']= DBSCAN(eps=5, min_samples=2).fit(for_bearing['angle'].to_numpy().reshape(-1,1)).labels_
    cur_group = for_bearing[for_bearing['group']>-1].groupby('group')
    for group in cur_group:
        data = group[1]
        if check_parallelism(data,is_test=False):
            buffers =data.buffer(cap_style=3,distance=30,join_style=3)
            # buffers.to_file(f'{path}/buffers.shp')
            one_buffer = buffers.unary_union
            simplify_poly =simplify(100)
            if isinstance(simplify_poly,MultiPolygon):
                pnt_list=[]
                for polygon in simplify_poly:
                    pnt_list.extend(polygon.exterior.coords[:-1])
            else:
                pnt_list = simplify_poly.exterior.coords[:-1]
            list_shp = [Point(item) for item in pnt_list]
            dis = 0
            for i, point in enumerate(list_shp):
                j =i
                for point2 in list_shp[i+1:]:
                    j+=1
                    if point.distance(point2)>dis:
                        max_dis=(i,j)
                        dis = point.distance(point2)
            dic_final['name'].append(name)
            dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]],pnt_list[max_dis[1]])))
            dic_final['highway'].append(the_largest_street)
            dic_final['bearing'].append(data['angle'].mean())
GeoDataFrame(dic_final,crs='epsg:3857').to_file(f'{path}/one_line.shp')

In [ ]:
dic_final = {'name':[],'geometry':[], 'highway':[],'bearing':[]}
for_time = len(my_groupby)
for i,street in enumerate(my_groupby):
    # Calculate time to run
    print(round(i/for_time*100,2))
    res= street[1]
    name = street[0]
    # groupby highway and angle
    groupbyhighway = res.groupby('highway')
    the_largest_street = res.groupby('highway').sum()['length'].sort_values(ascending=False).index[0]
    for_bearing_0 = groupbyhighway.get_group(the_largest_street).reset_index()
    for_bearing= for_bearing_0.dropna(subset= ['angle'],axis=0)
    if len(for_bearing)==0:
        continue
    for_bearing['group']= DBSCAN(eps=5, min_samples=2).fit(for_bearing['angle'].to_numpy().reshape(-1,1)).labels_
    cur_group = for_bearing[for_bearing['group']>-1].groupby('group')
    for group in cur_group:
        data = group[1]
        if check_parallelism(data,is_test=False):
            buffers =data.buffer(cap_style=3,distance=30,join_style=3)
            # buffers.to_file(f'{path}/buffers.shp')
            one_buffer = buffers.unary_union
            simplify_poly =simplify(100)
            if isinstance(simplify_poly,MultiPolygon):
                pnt_list=[]
                for polygon in simplify_poly:
                    pnt_list.extend(polygon.exterior.coords[:-1])
            else:
                pnt_list = simplify_poly.exterior.coords[:-1]
            list_shp = [Point(item) for item in pnt_list]
            dis = 0
            for i, point in enumerate(list_shp):
                j =i
                for point2 in list_shp[i+1:]:
                    j+=1
                    if point.distance(point2)>dis:
                        max_dis=(i,j)
                        dis = point.distance(point2)
            dic_final['name'].append(name)
            dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]],pnt_list[max_dis[1]])))
            dic_final['highway'].append(the_largest_street)
            dic_final['bearing'].append(data['angle'].mean())
GeoDataFrame(dic_final,crs='epsg:3857').to_file(f'{path}/one_line.shp')

In [7]:
name = 'Corso Regina Margherita'
new_geometry =['geometry_right','geometry_left']

res =my_groupby.get_group(name)
res.to_file(f'{path}/by_name.shp')
# groupby highway and angle
groupbyhighway = res.groupby('highway')
the_largest_street = res.groupby('highway').sum()['length'].sort_values(ascending=False).index[0]
for_bearing = groupbyhighway.get_group(the_largest_street).reset_index()
for_bearing['group']= DBSCAN(eps=5, min_samples=2).fit(for_bearing['angle'].to_numpy().reshape(-1,1)).labels_
cur_group = for_bearing[for_bearing['group']>-1].groupby('group')


In [10]:
for_bearing = groupbyhighway.get_group(the_largest_street).reset_index()
for_bearing['group']= DBSCAN(eps=5, min_samples=2).fit(for_bearing['angle'].to_numpy().reshape(-1,1)).labels_
cur_group = for_bearing[for_bearing['group']>-1].groupby('group')

In [62]:
dic_final = {'name':[],'geometry':[], 'highway':[],'bearing':[]}
for group in cur_group:
    data = group[1]
    if check_parallelism(data,is_test=True):
        buffers =data.buffer(cap_style=3,distance=30,join_style=3)
        buffers.to_file(f'{path}/buffers.shp')
        one_buffer = buffers.unary_union
        simplify_poly = one_buffer.simplify(100, preserve_topology=False)
        # dic = {'index':[0],'geometry':simplify_poly}
        # GeoDataFrame(dic,crs='epsg:3857').to_file(f'{path}/simplify_poly.shp')
        # dic = {'index':[0],'geometry':one_buffer}
        # GeoDataFrame(dic,crs='epsg:3857').to_file(f'{path}/one_buffer.shp')
        if isinstance(simplify_poly,MultiPolygon):
            pnt_list=[]
            for polygon in simplify_poly:
                pnt_list.extend(polygon.exterior.coords[:-1])
        else:
            pnt_list = simplify_poly.exterior.coords[:-1]
        list_shp = [Point(item) for item in pnt_list]
        dis = 0
        for i, point in enumerate(list_shp):
            j =i
            for point2 in list_shp[i+1:]:
                j+=1
                if point.distance(point2)>dis:
                    max_dis=(i,j)
                    dis = point.distance(point2)
        dic_final['name'].append(name)
        dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]],pnt_list[max_dis[1]])))
        dic_final['highway'].append(the_largest_street)
        dic_final['bearing'].append(data['angle'].mean())
GeoDataFrame(dic_final,crs='epsg:3857').to_file(f'{path}/one_line.shp')
        # if isinstance(one_buffer,Polygon):
        #     dic = {'index':[0],'geometry':one_buffer}
        #     GeoDataFrame(dic,crs='epsg:3857').to_file(f'{path}/one_buffer.shp')
        #     # my_centerline = Centerline(one_buffer)
        #     # dic = {'index':[0],'geometry':my_centerline}
        #     # GeoDataFrame(dic,crs='epsg:3857').to_file('new_line.shp')
        # else:
        #     GeoDataFrame(geometry=list(one_buffer.geoms),crs='epsg:3857').to_file(f'{path}/one_buffer.shp')
        # my_centerline = Centerline(one_buffer)
        # GeoDataFrame(geometry=list(my_centerline.geoms),crs='epsg:3857').to_file(f'{path}/new_line.shp')
        # break


C:\Users\Achituv\AppData\Local\Temp\ipykernel_17720\2792904332.py:15: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:


In [65]:
data['angle'].mean()

96.625

In [60]:
if isinstance(simplify_poly,MultiPolygon):
    for polygon in simplify_poly:
        points = polygon.exterior.coords[:-1]
        break
        # pnt_list.append(polygon.boundary.coords)

C:\Users\Achituv\AppData\Local\Temp\ipykernel_17720\616000580.py:2: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in simplify_poly:


In [61]:
points

[(857374.4367344987, 5632581.625295284),
 (857320.6824619013, 5632671.44128537),
 (857658.5255342015, 5632511.8748241)]

In [160]:
simplify_poly = one_buffer.simplify(100, preserve_topology=False)
dic = {'index':[0],'geometry':simplify_poly}
GeoDataFrame(dic,crs='epsg:3857').to_file(f'{path}/simplify_poly.shp')


In [182]:
pnt_list = list(simplify_poly.boundary.coords)
list_shp = [Point(item) for item in pnt_list]
dis = 0
max_dis = ()

In [199]:
pnt_list

[(848644.9687901939, 5629471.667221865),
 (848227.1217331432, 5629554.891152071),
 (849375.5342342065, 5629478.00315337),
 (848644.9687901939, 5629471.667221865)]

In [195]:
list_shp

In [209]:
for i, point in enumerate(list_shp):
    j =i
    for point2 in list_shp[i+1:]:
        j+=1
        if point.distance(point2)>dis:
            max_dis=(i,j)
            dis = point.distance(point2)
one_line = LineString(coordinates=(pnt_list[max_dis[0]],pnt_list[max_dis[1]]))
dic = {'index':[0],'geometry':one_line}
GeoDataFrame(dic,crs='epsg:3857').to_file(f'{path}/one_line.shp')

In [44]:
streets_to_clear_segment = []
new_geometry =['geometry_right','geometry_left']
for group in df_pro.groupby('name'):
    res =group[1]
    res_translate = res.copy()
    buffer_test = res['geometry'].buffer(cap_style=2,distance=30,join_style=3)
    res_translate['geometry_right'] = res_translate['geometry'].apply(lambda x:x.parallel_offset(35, 'right'))
    res_translate['geometry_left'] = res_translate['geometry'].apply(lambda x:x.parallel_offset(35, 'left'))

    if length_of_parallel(res_translate,'geometry_right') >0 or length_of_parallel(res_translate,'geometry_left') >0:
        print(f'{group[0]} has parallel segmetmts')
        streets_to_clear_segment.append(res)

streets_to_clear_segment_pd = pd.concat(streets_to_clear_segment)
streets_to_clear_segment_pd.to_file(f'{path}/streets_to_clear_segment.shp')

18 has parallel segmetmts
Area pedonale Vittime dell'Immigrazione has parallel segmetmts
Autostrada Serenissima has parallel segmetmts
Ciclabile Carducci - Biglieri has parallel segmetmts
Corso Achille Mario Dogliotti has parallel segmetmts
Corso Adriatico has parallel segmetmts
Corso Alberto Picco has parallel segmetmts
Corso Alcide De Gasperi has parallel segmetmts
Corso Alessandro Tassoni has parallel segmetmts
Corso Appio Claudio has parallel segmetmts
Corso Belgio has parallel segmetmts
Corso Benedetto Brin has parallel segmetmts
Corso Benedetto Croce has parallel segmetmts
Corso Bernardino Telesio has parallel segmetmts
Corso Bolzano has parallel segmetmts
Corso Bramante has parallel segmetmts
Corso Brianza has parallel segmetmts
Corso Cadore has parallel segmetmts
Corso Caduti sul lavoro has parallel segmetmts
Corso Caio Plinio has parallel segmetmts
Corso Cairoli has parallel segmetmts
Corso Carlo Luigi Farini has parallel segmetmts
Corso Carlo e Nello Rosselli has parallel seg

no parallel lines


In [53]:
new_data

,geometry,index_right
15436,"LINESTRING (857674.331 5626595.994, 857654.026...",15436


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:34: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._input_geometry = input_geometry
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:35: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._interpolation_distance = abs(interpolation_distance)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:40: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._min_x, self._min_y = self._get_reduced_coordinates()


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:34: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._input_geometry = input_geometry
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:35: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._interpolation_distance = abs(interpolation_distance)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:40: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._min_x, self._min_y = self._get_reduced_coordinates()


In [14]:
res.to_file('test.shp')